In [1]:
import duckdb as ddb
import pandas as pd

In [2]:
con = ddb.connect("../air_quality.db")
print(con)



In [ ]:
# This will allow us to take the result of our query 
# And turn it into a dataframe
# df = con.query("SELECT * FROM raw.air_quality WHERE parameter in ('so2', 'pm10', 'pm25')").to_df()
# Query from the presentation air_quality view
df = con.query("SELECT * FROM air_quality.presentation.air_quality").to_df()


df.head()


,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
0,947143,5790623,Wu Yee Children's Services-1011720,2024-02-13 15:06:13,37.72887,-122.3744,pm25,µg/m³,4.85,02,2024,2025-01-31 17:14:24.106
1,947143,5790623,Wu Yee Children's Services-1011720,2024-02-13 22:42:28,37.72887,-122.3744,pm25,µg/m³,2.22,02,2024,2025-01-31 17:14:24.106
2,947143,5790623,Wu Yee Children's Services-1011720,2024-02-14 10:42:18,37.72887,-122.3744,pm25,µg/m³,0.04,02,2024,2025-01-31 17:14:24.106
3,947143,5790623,Wu Yee Children's Services-1011720,2024-02-14 15:23:19,37.72887,-122.3744,pm25,µg/m³,0.07,02,2024,2025-01-31 17:14:24.106
4,947143,5790623,Wu Yee Children's Services-1011720,2024-02-14 23:03:43,37.72887,-122.3744,pm25,µg/m³,1.14,02,2024,2025-01-31 17:14:24.106


In [4]:
# Basic inspection of data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182660 entries, 0 to 182659
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   location_id         182660 non-null  int64         
 1   sensors_id          182660 non-null  int64         
 2   location            182660 non-null  object        
 3   datetime            182660 non-null  datetime64[us]
 4   lat                 182660 non-null  float64       
 5   lon                 182660 non-null  float64       
 6   parameter           182660 non-null  object        
 7   units               182660 non-null  object        
 8   value               182660 non-null  float64       
 9   month               182660 non-null  object        
 10  year                182660 non-null  int64         
 11  ingestion_datetime  182660 non-null  datetime64[us]
dtypes: datetime64[us](2), float64(3), int64(3), object(4)
memory usage: 16.7+ MB


In [5]:
# High level distribution 
df.describe()

,location_id,sensors_id,datetime,lat,lon,value,year,ingestion_datetime
count,182660.000000,1.826600e+05,182660,182660.000000,182660.000000,182660.000000,182660.0,182660
mean,468218.771910,3.643384e+06,2024-02-07 18:59:24.867179,37.788861,-122.311929,1.655570,2024.0,2025-01-31 17:10:26.498772
min,1311.000000,2.363000e+03,2024-01-01 08:02:23,37.482800,-122.518900,0.000000,2024.0,2025-01-31 17:06:57.386000
25%,2003.000000,2.556600e+04,2024-01-17 07:14:27.750000,37.743061,-122.411890,0.010017,2024.0,2025-01-31 17:07:53.382000
50%,8741.000000,4.598882e+06,2024-02-08 06:00:00,37.775905,-122.397800,0.027340,2024.0,2025-01-31 17:09:00.589000
75%,947136.000000,5.790711e+06,2024-02-26 04:00:00,37.797630,-122.237976,2.620000,2024.0,2025-01-31 17:13:08.571000
max,947143.000000,5.836916e+06,2024-04-01 07:00:00,38.102507,-121.784217,102.000000,2024.0,2025-01-31 17:14:27.640000
std,472124.175688,2.565666e+06,NaN,0.112626,0.163506,3.110143,0.0,NaN


In [6]:
# Perform describe method on string like data. Note: That's an O not a zero
df.describe(include='O')

,location,parameter,units,month
count,182660,182660,182660,182660
unique,32,6,2,3
top,San Rafael-2002,pm25,ppm,01
freq,10455,63174,119486,77642


In [7]:
df.columns

Index(['location_id', 'sensors_id', 'location', 'datetime', 'lat', 'lon',
       'parameter', 'units', 'value', 'month', 'year', 'ingestion_datetime'],
      dtype='object')

In [8]:
# Check for duplicates. We want most of the columns, but we don't want, for example, 'location'. Because that will have duplicates. Note: This returned an empty dataframe. That means we don't have any duplicates
df[df.duplicated(subset=['location_id', 'datetime', 'parameter', 'units', 'value'])]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime


In [59]:
df[df['location_id'].eq(2009) & 
   df['datetime'].eq('2024-01-26 10:00:00') & 
   df['parameter'].eq('pm25') & 
   df['units'].eq('µg/m³') & 
   df['value'].eq(14.0000)]


,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
67120,2009,3569,San Francisco-2009,2024-01-26 10:00:00,37.7658,-122.3978,pm25,µg/m³,14.0,01,2024,2025-01-29 13:49:36.791
447769,2009,3569,San Francisco-2009,2024-01-26 10:00:00,37.7658,-122.3978,pm25,µg/m³,14.0,01,2024,2025-01-31 13:45:05.657
451081,2009,3569,San Francisco-2009,2024-01-26 10:00:00,37.7658,-122.3978,pm25,µg/m³,14.0,01,2024,2025-01-31 14:36:02.372


In [25]:
# Keep the first occurence and remove the rest
df_drop_duplicates = df.drop_duplicates(subset=['location_id', 'datetime', 'parameter', 'units', 'value'])


In [26]:
# Check for duplicates. We want most of the columns, but we don't want, for example, 'location'. Because that will have duplicates. Note: This returned an empty dataframe. That means we don't have any duplicates
df_drop_duplicates[df_drop_duplicates.duplicated(subset=['location_id', 'datetime', 'parameter', 'units', 'value'])]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime


In [28]:
df_drop_duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Index: 432738 entries, 0 to 444461
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   location_id         432738 non-null  int64         
 1   sensors_id          432738 non-null  int64         
 2   location            432738 non-null  object        
 3   datetime            432738 non-null  datetime64[us]
 4   lat                 432738 non-null  float64       
 5   lon                 432738 non-null  float64       
 6   parameter           432738 non-null  object        
 7   units               432738 non-null  object        
 8   value               432738 non-null  float64       
 9   month               432738 non-null  object        
 10  year                432738 non-null  int64         
 11  ingestion_datetime  432738 non-null  datetime64[us]
dtypes: datetime64[us](2), float64(3), int64(3), object(4)
memory usage: 42.9+ MB


In [24]:
print(df.head())

   location_id  sensors_id            location            datetime        lat  \
0         1311        2366  Laney College-1311 2024-01-01 09:00:00  37.793624   
1         1311        2366  Laney College-1311 2024-01-01 10:00:00  37.793624   
2         1311        2366  Laney College-1311 2024-01-01 11:00:00  37.793624   
3         1311        2366  Laney College-1311 2024-01-01 12:00:00  37.793624   
4         1311        2366  Laney College-1311 2024-01-01 13:00:00  37.793624   

          lon parameter  units  value month  year      ingestion_datetime  
0 -122.263376      pm25  µg/m³    4.0    01  2024 2025-01-29 13:46:57.003  
1 -122.263376      pm25  µg/m³    6.0    01  2024 2025-01-29 13:46:57.003  
2 -122.263376      pm25  µg/m³    7.0    01  2024 2025-01-29 13:46:57.003  
3 -122.263376      pm25  µg/m³   10.0    01  2024 2025-01-29 13:46:57.003  
4 -122.263376      pm25  µg/m³   11.0    01  2024 2025-01-29 13:46:57.003  


In [11]:
# Want to check if the different parameters have the same number of measurements or do they differ. Note: This returned only pm25.
df.groupby(by='parameter', as_index=False).count()

,parameter,location_id,sensors_id,location,datetime,lat,lon,units,value,month,year,ingestion_datetime
0,bc,2826,2826,2826,2826,2826,2826,2826,2826,2826,2826,2826
1,co,15613,15613,15613,15613,15613,15613,15613,15613,15613,15613,15613
2,no,19469,19469,19469,19469,19469,19469,19469,19469,19469,19469,19469
3,no2,60159,60159,60159,60159,60159,60159,60159,60159,60159,60159,60159
4,nox,19469,19469,19469,19469,19469,19469,19469,19469,19469,19469,19469
5,o3,16241,16241,16241,16241,16241,16241,16241,16241,16241,16241,16241
6,pm1,51604,51604,51604,51604,51604,51604,51604,51604,51604,51604,51604
7,pm10,43979,43979,43979,43979,43979,43979,43979,43979,43979,43979,43979
8,pm25,66176,66176,66176,66176,66176,66176,66176,66176,66176,66176,66176
9,relativehumidity,7536,7536,7536,7536,7536,7536,7536,7536,7536,7536,7536


In [13]:
# Check the negative values
df[df["value"] < 0]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
2766,1311,2366,Laney College-1311,2024-02-04 14:00:00,37.793624,-122.263376,pm25,µg/m³,-1.00000,02,2024,2025-01-29 13:47:00.395
2767,1311,2366,Laney College-1311,2024-02-04 15:00:00,37.793624,-122.263376,pm25,µg/m³,-1.00000,02,2024,2025-01-29 13:47:00.395
2772,1311,2366,Laney College-1311,2024-02-04 20:00:00,37.793624,-122.263376,pm25,µg/m³,-1.00000,02,2024,2025-01-29 13:47:00.395
4758,1311,2366,Laney College-1311,2024-02-26 10:00:00,37.793624,-122.263376,pm25,µg/m³,-1.00000,02,2024,2025-01-29 13:47:00.395
5760,1311,2366,Laney College-1311,2024-03-04 13:00:00,37.793624,-122.263376,pm25,µg/m³,-1.00000,03,2024,2025-01-29 13:47:03.510
...,...,...,...,...,...,...,...,...,...,...,...,...
417293,947142,5790620,Bayshore and Industrial -1011727,2024-02-27 17:20:11,37.738050,-122.407480,no2,ppm,-0.00569,02,2024,2025-01-29 13:55:43.240
425205,947143,5790616,Wu Yee Children's Services-1011720,2024-01-02 21:25:35,37.728870,-122.374400,no2,ppm,-0.00256,01,2024,2025-01-29 13:55:50.770
425206,947143,5790616,Wu Yee Children's Services-1011720,2024-01-02 21:43:07,37.728870,-122.374400,no2,ppm,-0.00366,01,2024,2025-01-29 13:55:50.770
425207,947143,5790616,Wu Yee Children's Services-1011720,2024-01-02 22:00:40,37.728870,-122.374400,no2,ppm,-0.00310,01,2024,2025-01-29 13:55:50.770


In [31]:
con.close()

### Explanation for the ROW_NUMBER() function 

Of course! Think of the `ROW_NUMBER()` function as a way to assign a unique number to each row in a group of data, based on a specific sorting order. Here’s an easy way to break it down:

### Imagine a Stack of Papers
- You have a stack of papers for different locations, sensors, and dates.
- You organize the papers by location, then by sensor, and then by date and pollutant (this is the **PARTITION BY** part—it’s how the rows are grouped together).
- Within each group, you sort the papers by the time they were received, starting with the most recent at the top (this is the **ORDER BY ingestion_datetime DESC** part).

### Now Assign Numbers
- After organizing, you go through each group and number the papers: 1 for the first, 2 for the second, and so on. 
- The first paper in each group (the one with the most recent `ingestion_datetime`) gets the number `1`.

### What Does This Look Like in SQL?
Here’s an example:  
If you have a table with data like this:

| location_id | sensors_id | datetime   | parameter | ingestion_datetime    |
|-------------|------------|------------|-----------|-----------------------|
| 101         | A          | 2025-01-01 | co        | 2025-01-02 10:00:00  |
| 101         | A          | 2025-01-01 | co        | 2025-01-02 08:00:00  |
| 102         | B          | 2025-01-01 | no2       | 2025-01-03 09:00:00  |

After applying `ROW_NUMBER()`:

| location_id | sensors_id | datetime   | parameter | ingestion_datetime    | rn |
|-------------|------------|------------|-----------|-----------------------|----|
| 101         | A          | 2025-01-01 | co        | 2025-01-02 10:00:00  |  1 |
| 101         | A          | 2025-01-01 | co        | 2025-01-02 08:00:00  |  2 |
| 102         | B          | 2025-01-01 | no2       | 2025-01-03 09:00:00  |  1 |

### Why is This Useful?
Once rows are numbered, you can easily pick the "best" or "latest" row (like `rn = 1`) within each group, leaving behind duplicates or older data. 

Does this make it easier to understand? 😊

* This does help, but if we have duplicate rns (row numbers), how is rn used as an identifier for say no2. Because we have now two groups of no2 with the same rn. 

Great observation! Here's the key idea: **`ROW_NUMBER()` ensures there are no duplicates within a single group**. However, there can be multiple groups (e.g., different `datetime` values for `no2`), and each group's rows will have their own `rn` starting from `1`.

Let’s dive into why this is not a problem:

### Unique Row Numbers Are Within Each Group
The `ROW_NUMBER()` function assigns `rn` **separately for each group** (as defined by the `PARTITION BY` clause). So, if you have two different groups of `no2` (say, one for `datetime = 2025-01-01` and another for `datetime = 2025-01-02`), each group starts counting its rows independently. This means:

- The **row numbers are unique within a group**.
- You can safely use `rn = 1` to identify the most recent record *within that specific group*.

### Why Duplicates Across Groups Aren’t an Issue
When you query the data, the combination of the `GROUP BY` fields (`location_id`, `sensors_id`, `datetime`, and `parameter`) ensures that each group is unique. Let’s revisit our example:

| location_id | sensors_id | datetime   | parameter | ingestion_datetime    | rn |
|-------------|------------|------------|-----------|-----------------------|----|
| 102         | B          | 2025-01-01 | no2       | 2025-01-03 09:00:00  |  1 |
| 102         | B          | 2025-01-02 | no2       | 2025-01-04 10:00:00  |  1 |

Here’s what happens:
1. `location_id = 102`, `sensors_id = B`, `datetime = 2025-01-01`, `parameter = no2`:  
   This is one group. The most recent record here has `rn = 1`.

2. `location_id = 102`, `sensors_id = B`, `datetime = 2025-01-02`, `parameter = no2`:  
   This is a **different group**, so the row with the most recent `ingestion_datetime` in this group also has `rn = 1`.

When querying for `rn = 1`, you’re selecting **the most recent record for each unique group**. The groups are already distinct because of `location_id`, `sensors_id`, `datetime`, and `parameter`.

### Summary
While there can be multiple `rn = 1` rows across groups, **they belong to separate groups**, making it easy to distinguish them. When you query the view, the unique combination of fields like `location_id`, `sensors_id`, `datetime`, and `parameter` ensures there’s no confusion. 

This means `rn` is effectively a local identifier within each group, not a global one, and that’s why duplicates across groups are perfectly fine.